In [ ]:
!pip install datasets transformers rouge_score nltk evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=588d4a61b944c7fcffd9e5ea7ad6ae260a2147a0a83970fe7ac32353093b23f9
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 9.1 MB/s eta 0:00:00


In [ ]:
import transformers
import pandas as pd
import torch
#from nlp import load_dataset, load_metric

In [ ]:
from datasets import load_dataset

# Specify the path to the dataset file
data_dir = '/content/drive/MyDrive/wikihow_data/'

# Load the WikiHow dataset
dataset = load_dataset('wikihow', 'all', data_dir=data_dir)


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/157252 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5599 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5577 [00:00<?, ? examples/s]

In [ ]:
import nltk
nltk.download('punkt')
import string
from transformers import AutoTokenizer

model_checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'headline', 'title'],
        num_rows: 157252
    })
    validation: Dataset({
        features: ['text', 'headline', 'title'],
        num_rows: 5599
    })
    test: Dataset({
        features: ['text', 'headline', 'title'],
        num_rows: 5577
    })
})

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")

    labels = tokenizer(text_target=examples["headline"], max_length=1024, truncation=True,padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/157252 [00:00<?, ? examples/s]

Map:   0%|          | 0/5599 [00:00<?, ? examples/s]

Map:   0%|          | 0/5577 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.save_to_disk("/content/drive/MyDrive/T5small_wikihow_tokenized")

Saving the dataset (0/6 shards):   0%|          | 0/157252 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5599 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5577 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk

train_tokenized_dataset = load_from_disk("/content/drive/MyDrive/wikihow_data/T5small_train_tokenized")
test_tokenized_dataset = load_from_disk("/content/drive/MyDrive/wikihow_data/T5small_test_tokenized")
#validation_tokenized_dataset = load_from_disk("/content/drive/MyDrive/T5small_validation_tokenized")

# tokenized_dataset = load_from_disk("/content/drive/MyDrive/T5small_wikihow_tokenized")

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_checkpoint,max_length=200)

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch

In [ ]:
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last two layers
for param in model.encoder.block[-1].layer[-1:].parameters():
    param.requires_grad = True

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/wikihow_t5small_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=test_tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
output_dir = "/content/drive/MyDrive/T5fine_tuned_model"
trainer.save_model(output_dir)

In [ ]:
text = dataset['train'][1]['text']
summary = dataset['train'][1]['headline']

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/Prashanth-1998/wikihow_t5small_model/tree/main/'

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

summarizer = pipeline("summarization", model="Prashanth-1998/wikihow_t5small_model")
summarizer(text)

Token indices sequence length is longer than the specified maximum sequence length for this model (821 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': 'Create a NeoPopRealist mural. Measure the actual area to be painted. Use a scaled sketch to measure the distances and locations of key elements of your subject. Create the NeoPoprealist ART. Use the lines to create the image shape and sections.'}]

In [ ]:
print(summary)

Create a sketch in the NeoPopRealist manner of the future mural on a small piece of paper 8"x10" using the black ink pen.
Prepare to create your NeoPopRealist mural.
Prepare your paint.
Begin your project with a design.
Produce a scaled down version of your finished mural.
Prepare the wall to be painted.
After you have primed the surface, measure the wall.
Paint in the base coat of the background.
Allow the background and base coats to dry.
Draw the lines, then fill the appeared section with different repetitive patterns (examine the images above).
Paint patterns with brushes of suitable size for the particular portion of work you are painting.
Clean up the lines and shapes as needed.
Seal the mural if needed.
Be inspired and it will help you succeed!
